# Preamble

In [1]:
from flair.datasets import ColumnCorpus # Used to load and preprocess text data for NLP tasks
from flair.embeddings import FlairEmbeddings # Representations of words in vector form
from flair.embeddings import CharacterEmbeddings # Representations of words in vector form
from flair.embeddings import TokenEmbeddings # Representations of words in vector form
from flair.embeddings import StackedEmbeddings # Representations of words in vector form
from flair.models import SequenceTagger # sequence tagging, NER or POS
from flair.trainers import ModelTrainer # Training Flair models
from typing import List
import numpy as np
import os
import torch
import random

/home/pclinux/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# files on: https://groups.uni-paderborn.de/wdqa/causenet/flair-models/texts/
PATH_SPOTTING_DATASET = "../../data/concept-spotting/sentences/"
PATH_FLAIR_FOLDER = "../../data/flair-models/sentences/"

# Sentence-Spotter: Training

In [3]:
def set_seed(seed):
    # For reproducibility
    # (https://pytorch.org/docs/stable/notes/randomness.html)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    torch.backends.cudnn.deterministic = True # cuDNN operations will produce the same results on the GPU across runs
    torch.backends.cudnn.benchmark = False #  When enabled, cuDNN may dynamically choose the best algorithm for convolution operations, which can result in slightly different numerical results across runs.

In [4]:
# data preperation for NLP, defning the structure of the dataset, type of tagging, dictionary for tags
# Mapping column indices (0, 1, 2) to column names ('text', 'pos', 'chunk_BIO'), the structure of the data set
# text, part of speech, containing BIO (Begin,Inside,Outside) - beginning, inside and end of the sentence
columns = {0: 'text', 1: 'pos', 2: 'chunk_BIO'} 
tag_type = "chunk_BIO"
corpus = ColumnCorpus(PATH_SPOTTING_DATASET, columns) # 
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type) # Storing the tags/bio-chunks
print(corpus)

2023-10-07 17:19:08,649 Reading data from ../../data/concept-spotting/sentences
2023-10-07 17:19:08,651 Train: ../../data/concept-spotting/sentences/train.txt
2023-10-07 17:19:08,652 Dev: ../../data/concept-spotting/sentences/dev.txt
2023-10-07 17:19:08,654 Test: ../../data/concept-spotting/sentences/test.txt


/tmp/ipykernel_11762/1214949526.py:7: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type) # Storing the tags/bio-chunks


Corpus: 583 train + 127 dev + 122 test sentences


In [10]:
set_seed(42)
# defining a list of embeddings to be used in the model
# CharacterEmbeddings : characters in the word, capturing subword information
# FlairEmbeddings : pretrained contextual embeddings - based on forward & backward pass LM trained on news
embedding_types: List[TokenEmbeddings] = [
    CharacterEmbeddings(), 
    FlairEmbeddings('news-forward'), 
    FlairEmbeddings('news-backward')]

# Stacking the embeddings types created before.
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
set_seed(42)

# Extracting important parts of sentences
tagger: SequenceTagger = SequenceTagger(hidden_size=64,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True,
                                        dropout=0.25,
                                        rnn_layers=2)
set_seed(42)
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

2023-10-07 17:46:04,128 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>


In [11]:
set_seed(42)
result = trainer.train(PATH_FLAIR_FOLDER,
                       learning_rate=0.2,
                       mini_batch_size=32,
                       max_epochs=20,
                       shuffle=True,
                       num_workers=0)

2023-10-07 17:46:05,913 ----------------------------------------------------------------------------------------------------
2023-10-07 17:46:05,914 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (dropout): Dropout(p=0.25, inplace=False)
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4146, out_features=4146, bias=True)
  (rnn): LSTM(4146, 64, num_layers=2, batch_first

100%|██████████| 4/4 [00:02<00:00,  1.49it/s]

2023-10-07 17:46:28,140 Evaluating as a multi-label problem: False
2023-10-07 17:46:28,158 DEV : loss 9.036890347147164e-09 - f1-score (micro avg)  0.0
2023-10-07 17:46:28,182 BAD EPOCHS (no improvement): 0
2023-10-07 17:46:28,186 saving best model


2023-10-07 17:46:29,277 ----------------------------------------------------------------------------------------------------
2023-10-07 17:46:30,788 epoch 2 - iter 1/19 - loss 0.00000000 - time (sec): 1.51 - samples/sec: 636.17 - lr: 0.200000
2023-10-07 17:46:31,790 epoch 2 - iter 2/19 - loss 0.00000001 - time (sec): 2.51 - samples/sec: 770.25 - lr: 0.200000
2023-10-07 17:46:32,677 epoch 2 - iter 3/19 - loss 0.00000001 - time (sec): 3.40 - samples/sec: 813.88 - lr: 0.200000
2023-10-07 17:46:33,763 epoch 2 - iter 4/19 - loss 0.00000001 - time (sec): 4.48 - samples/sec: 833.31 - lr: 0.200000
2023-10-07 17:46:35,063 epoch 2 - iter 5/19 - loss 0.00000001 - time (sec): 5.78 - samples/sec: 836.09 - lr: 0.200000
2023-10-07 17:46:35,942 epoch 2 - iter 6/19 - loss 0.00000001 - time (sec): 6.66 - samples/sec: 847.50 - lr: 0.200000
2023-10-07 17:46:37,080 epoch 2 - iter 7/19 - loss 0.00000000 - time (sec): 7.80 - samples/sec: 847.54 - lr: 0.200000
2023-10-07 17:46:38,159 epoch 2 - iter 8/19 - los

100%|██████████| 4/4 [00:02<00:00,  1.52it/s]

...

100%|██████████| 4/4 [00:02<00:00,  1.45it/s]

2023-10-07 17:53:20,072 Evaluating as a multi-label problem: False
2023-10-07 17:53:20,089 DEV : loss 9.036890347147164e-09 - f1-score (micro avg)  0.0
2023-10-07 17:53:20,128 BAD EPOCHS (no improvement): 3


2023-10-07 17:53:20,672 ----------------------------------------------------------------------------------------------------
2023-10-07 17:53:23,181 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>


100%|██████████| 4/4 [00:02<00:00,  1.50it/s]

2023-10-07 17:53:26,065 Evaluating as a multi-label problem: False
2023-10-07 17:53:26,086 0.0	0.0	0.0	0.0
2023-10-07 17:53:26,087 
Results:
- F-score (micro) 0.0
- F-score (macro) 0.0
- Accuracy 0.0

By class:
              precision    recall  f1-score   support

           C     0.0000    0.0000    0.0000     351.0

   micro avg     0.0000    0.0000    0.0000     351.0
   macro avg     0.0000    0.0000    0.0000     351.0
weighted avg     0.0000    0.0000    0.0000     351.0

2023-10-07 17:53:26,088 ----------------------------------------------------------------------------------------------------


In [12]:
result['test_score']

0.0

In [8]:
assert result['test_score'] == 0.6466

AssertionError: 